### PM2.5 Data (Berkeley Earth)

- Bangkok 13.754,100.5014
- hourly data since 2016-03-03 (use this one)
   - http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/Bangkok/Bangkok.txt
- Air4Thai (only latest 2 months) 50t = Chula hospital station
   - http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID=50t&param=PM25&type=hr&sdate=2018-10-01&edate=2019-01-17&stime=00&etime=24
   - http://air4thai.pcd.go.th/webV2/history/

### Weather Data (wunderground.com)
- Patumwan Demonstration School weather station, since 2013-12-17 
   - https://www.wunderground.com/personal-weather-station/dashboard?ID=IBANGKOK26#history/tdata/s20140210/e20140210/mdaily

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import pandas as pd
from datetime import timedelta, date
from tqdm import tqdm

start_date = date(2016, 3, 3)
end_date = date(2019, 1, 28)

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def daterange(start_date, end_date):
    for n in tqdm(range(int ((end_date - start_date).days))):
        yield start_date + timedelta(n)

write_header = 1
write_header_file = 1 # change to 0 to continue without writing header to csv
for single_date in daterange(start_date, end_date):
    df_daily = pd.DataFrame()
    ymd = single_date.strftime("%Y%m%d")
    url_test = "https://api-ak.wunderground.com/api/606f3f6977348613/history_"+ymd+"null/units:metric/v:2.0/q/pws:IBANGKOK26.json"
    req = requests.get(url_test)
    js = json.loads(req.text)
    if js['history']['days'] == []:
        print("empty data on",ymd)
        continue
    for ob in js['history']['days'][0]['observations']:
        ob['date'] = ob['date']['iso8601']
        df_row = pd.DataFrame(ob, index=[0]) 
        df_row.index = df_row['date']
        df_row = df_row.drop(columns=['date'])
        if write_header == 1:
            df_daily = df_row
            write_header = 0
        else:
            df_daily = df_daily.append(df_row)

    with open('patumwan_weather.csv','a+') as fd:
        fd.write(df_daily.to_csv(header=write_header_file))
        write_header_file = 0


100%|██████████| 10/10 [00:37<00:00,  3.12s/it]
